In [0]:
!pip3 install numpy 
!pip3 install scipy 
!pip3 install opencv-python 
!pip3 install pillow 
!pip3 install matplotlib 
!pip3 install h5py 
!pip3 install keras 
!pip3 install https://github.com/OlafenwaMoses/ImageAI/releases/download/2.0.3/imageai-2.0.3-py3-none-any.whl 
!pip3 install pandas
import pandas as pd


     |████████████████████████████████| 153kB 610kB/s 


This mounts the program on your google drive. It should be adjusted if running on AWS

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


This ensures the GPU is ready to run

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


This imports necessary libraries for image ai. The last line is important if running on Collab, but might need to be changed if running in AWS

In [0]:
from imageai.Prediction import ImagePrediction
from imageai.Detection import VideoObjectDetection
from imageai.Prediction.Custom import CustomImagePrediction
import csv
import os
# mount gdrive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using TensorFlow backend.


##  The next two cells are used to running the model 


In [0]:
prediction = CustomImagePrediction()
prediction.setModelTypeAsResNet()
prediction.setModelPath("/content/drive/My Drive/Cleaned_Data/finished models/model_ex-097_acc-0.966667.h5")
prediction.setJsonPath("/content/drive/My Drive/Cleaned_Data/output/json/model_class.json")
prediction.loadModel(num_objects=2)

W0807 19:31:11.114903 139921994033024 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
file_path = "/content/drive/My Drive/AudioNet-master/tf_files/data_audio/" #Change this file pathway to the folder containing the audio images you want to analyze before running
file_list = []
record_anomalies = True
for subdir, dirs, files in os.walk(file_path):
    for file in files:
        file_list.append(os.path.join(subdir, file))
results = []
for file in file_list:
  if ".jpg" in str(file):
    predictions, probabilities = prediction.predictImage(os.path.join(file_path,file), result_count=2)
    print(file, predictions, probabilities)
    results.append((file, predictions, probabilities))
with open('results.csv', 'w', newline='') as csvfile: #Change 'results.csv' to the name that you want the results to be saved under
    resultswriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    anomaly = ""
    for result in results:
      if not record_anomalies:
        if result[1][0] == "Three":
          anomaly = "Anomaly" if float(result[2][0]) >= 80.0 else "Not Anomaly"
        elif result[1][1] == "Three":
          anomaly = "Anomaly" if float(result[2][1]) >= 80.0 else "Not Anomaly"
        resultswriter.writerow([result[0], result[1], result[2], anomaly])
      # if we wnat to only record anomalies 
      else: 
        if result[1][0] == "Three" and float(result[2][0]) >= 80.0:
          resultswriter.writerow([result[0], result[1], result[2]])


/content/drive/My Drive/AudioNet-master/tf_files/data_audio/4/GP030412chunk457.jpg ['One', 'Three'] ['99.99979734420776', '0.0001975498889805749']
/content/drive/My Drive/AudioNet-master/tf_files/data_audio/4/GP030412chunk67.jpg ['One', 'Three'] ['99.99691247940063', '0.003087389632128179']
/content/drive/My Drive/AudioNet-master/tf_files/data_audio/4/GP030412chunk291.jpg ['One', 'Three'] ['99.97692704200745', '0.02307776449015364']
/content/drive/My Drive/AudioNet-master/tf_files/data_audio/4/GP030412chunk287.jpg ['One', 'Three'] ['99.58311915397644', '0.41687809862196445']
/content/drive/My Drive/AudioNet-master/tf_files/data_audio/4/GP030412chunk458.jpg ['One', 'Three'] ['99.97653365135193', '0.02346507244510576']
/content/drive/My Drive/AudioNet-master/tf_files/data_audio/4/GP030412chunk493.jpg ['One', 'Three'] ['96.8302845954895', '3.1697187572717667']
/content/drive/My Drive/AudioNet-master/tf_files/data_audio/4/GP030412chunk598.jpg ['One', 'Three'] ['99.96984004974365', '0.03015

KeyboardInterrupt: ignored

## The cell below trains the model

```
# This is formatted as code
```



In [0]:
from imageai.Prediction.Custom import ModelTraining
model_trainer = ModelTraining()
model_trainer.setModelTypeAsResNet()
model_trainer.setDataDirectory("/content/drive/My Drive/Cleaned_Data/output")
model_trainer.trainModel(num_objects=2, num_experiments=100, enhance_data=True, batch_size=5, show_network_summary=True)


In [0]:
results = []
with open('/content/drive/My Drive/results.csv', newline='') as csvfile:
      csvreader = csv.reader(csvfile, delimiter=',')
      for row in csvreader:
        if(row[3] == 'Anomaly'): 
          results.append(row)
          print(row)
with open('results.csv', 'w', newline='') as csvfile:
    resultswriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for result in results: 
      resultswriter.writerow([result[0], result[1], result[2]])


The two cells below are effectively scrapwork. They were used to test formating, which is implemented in the other notebook to run the model


In [0]:
filename = os.path.basename(value)
            for index in range(len(filename)):
                if not filename[index].isdigit():
                    break
                clip_number += filename[index]
#             print(clip_number)
            match = re.search("[^k][\d]+(?=\.)", filename)
            mod_time = ((int(clip_number) - starting_clip) * total_clip_time)
#             print(mod_time)

In [0]:
data = pd.read_csv('results')
data.columns = ['filePath','breakdown', 'percentage', 'anomaly']
data['urlNum']= re.search("[^k][\d]+(?=\.)", data['filePath'])
data=data.sort_values(by=['urlNum'])
data.to_csv('testResults.csv')

FileNotFoundError: ignored